In [1]:
from MAESeqModule.MAESeq_utils import dataloader,get_dict, seq_data_to_onehot
import numpy as np
DICT_EXIST = True
DICT_PATH_CHAR2INT = 'dict/char2int.npy'
DICT_PATH_INT2CHAR = 'dict/int2char.npy'
DATA_PATH = 'dataset/scop_fa_represeq_lib_latest.fa'

seq_list, max_len = dataloader(
    file= DATA_PATH,
    len_data=10000, max_len_percintile=75)
max_len = 300 # 在这里统一一下

if DICT_EXIST:
    dict_char2int = np.load(DICT_PATH_CHAR2INT, allow_pickle = True).item()
    dict_int2char = np.load(DICT_PATH_INT2CHAR,allow_pickle = True).item()
else:
    dict_char2int, dict_int2char = get_dict(seq_list)
    np.save(DICT_PATH_CHAR2INT, dict_char2int)
    np.save(DICT_PATH_INT2CHAR, dict_int2char)

onehot_data = seq_data_to_onehot(seq_list,dict_char2int,max_len)
dimension = len(dict_char2int)

print("Length of seq is {}".format(max_len))
print("Dimension is {}".format(dimension))


2023-02-28 22:59:57.867937: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-28 23:00:00.280517: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-28 23:00:00.280537: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-28 23:00:05.226785: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

Length of seq is 300
Dimension is 25


In [2]:
import numpy as np
onehot_train, onehot_val,onehot_test =np.split(onehot_data,[
        int(len(onehot_data)*0.8), 
        int(len(onehot_data)*0.95)])
print(onehot_train.shape)
print(onehot_val.shape)
print(onehot_test.shape)



(8000, 300, 25)
(1500, 300, 25)
(500, 300, 25)


In [24]:
# from MAESeqModule.MAESeq_utils import mask_onehot_matrix
def mask_onehot_matrix(onehot_data, mask_rate = 0.2):
    # To input the whole data
    len_data = onehot_data.shape[0]
    res = onehot_data.copy()
    val_len = np.sum(np.sum(onehot_data, axis=2),axis=1)
    mask_len = (val_len * mask_rate).astype(np.int32)
    right_limit = val_len-mask_len
    start_point = np.random.randint(right_limit)
    end_point = start_point+mask_len
    # len_seq = onehot_data.shape[1]
    # len_mask = int(mask_rate*len_seq)
    # for single_matrix in res:
    #     mask_choose = np.random.choice(len_seq,len_mask,replace=False)
    #     single_matrix[mask_choose,:]=0
    for _ in range(len_data):
        res[_,start_point[_]:end_point[_],:] = 0.
    return res

onehot_train_mask = mask_onehot_matrix(onehot_train, 0.9)

i = 40
print(np.sum(onehot_train[i]))
print(np.sum(onehot_train_mask[i]))

117.0
12.0


In [5]:
from MAESeqModule.MAESeq_model import AutoencoderGRU, ReconstructRateVaried,my_loss_entropy
import keras
import tensorflow as tf

autoencoder = keras.models.load_model('/geniusland/home/qufuchuan/trained_model/',custom_objects={
    'my_loss_entropy':my_loss_entropy, 
    'ReconstructRateVaried':ReconstructRateVaried})


2023-02-28 18:24:49.746184: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-28 18:24:58.966031: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 62777 MB memory:  -> device: 0, name: NVIDIA A100 80GB PCIe, pci bus id: 0000:12:00.0, compute capability: 8.0
2023-02-28 18:24:58.971676: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 70029 MB memory:  -> device: 1, name: NVIDIA A100 80GB PCIe, pci bus id: 0000:13:00.0, compute capability: 8.0
2023-02-28 18:24:58.973409: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/tas

In [7]:
from MAESeqModule.MAESeq_utils import mask_onehot_matrix, evaluate_per_mask_rate
import numpy as np
import pandas as pd

res = evaluate_per_mask_rate(autoencoder=autoencoder, onehot_test = onehot_test)
res



2023-02-28 18:27:40.430446: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8100
2023-02-28 18:27:44.108042: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


Mask 0.00    0.894837
Mask 0.05    0.854087
Mask 0.10    0.811198
Mask 0.15    0.771023
Mask 0.20    0.727425
Mask 0.25    0.684377
Mask 0.30    0.641476
Mask 0.35    0.599687
Mask 0.40    0.557495
Mask 0.45    0.509900
Mask 0.50    0.466034
Mask 0.55    0.420700
Mask 0.60    0.376723
Mask 0.65    0.328652
Mask 0.70    0.282585
Mask 0.75    0.237447
Mask 0.80    0.190354
Mask 0.85    0.149115
Mask 0.90    0.106006
Mask 0.95    0.072003
Mask 1.00    0.043708
dtype: float64